# Integrating the TOV equations to give initial conditions for Rotating Neutron Stars

Unless stated otherwise, equations are given in geometrized units where $G=c=1$.

## The TOV equations in Schwarzschild coordinates

Start with a general spherically symmetric metric in Schwarzschild radial coordinates:
\begin{align}
	\rm{d}s^2 &= -e^{2\Phi(\tilde{r})}{\rm d}t^2 + \frac1{1-\frac{2m(\tilde{r})}{\tilde{r}}}{\rm d}\tilde{r}^2+\tilde{r}^2{\rm d}^2\Omega\qquad\text{for}\quad 0\le \tilde{r}\le \tilde{R}, \\
		&= -\left(1-\frac{2M}{\tilde{r}}\right){\rm d}t^2 + \frac1{\left(1-\frac{2M}{\tilde{r}}\right)}{\rm d}\tilde{r}^2+\tilde{r}^2{\rm d}^2\Omega\qquad\text{for}\quad \tilde{R}\le \tilde{r},
\end{align}
where $M$ is the mass of the Neutron Star and $\tilde{R}$ is the Schwarzschild coordinate radius of the Neutron Star.  We also use the notation that ${\rm d}^2\Omega= {\rm d}\theta^2 + \sin^2\theta{\rm d}\phi^2$ is the metric on the unit 2-sphere.  Einstein's equations, and the equation for hydrostatic equilibrium of a perfect fluid, yield the Tolman-Oppenheimer-Volkoff (TOV) equations for the interior structure of a spherical star:
\begin{align}
	\frac{{\rm d} m(\tilde{r})}{{\rm d}\tilde{r}} &= 4\pi \tilde{r}^2\epsilon(\tilde{r}) &&\text{with}\qquad m(\tilde{r}=0)=0,\\
	\frac{{\rm d} P(\tilde{r})}{{\rm d}\tilde{r}} &= -\frac{[\epsilon(\tilde{r})+P(\tilde{r})][m(\tilde{r})+4\pi \tilde{r}^3P(\tilde{r})]}{\tilde{r}[\tilde{r}-2m(\tilde{r})]} &&\text{with}\qquad\left\{\begin{array}{l}P(\tilde{r}=0)=P_c,\\
	P(\tilde{r}=\tilde{R})=0,\end{array}\right.\\
	\frac{{\rm d} \Phi(\tilde{r})}{{\rm d}\tilde{r}} &= \frac{m(\tilde{r})+4\pi \tilde{r}^3P(\tilde{r})}{\tilde{r}[\tilde{r}-2m(\tilde{r})]} &&\text{with}\qquad \Phi(\tilde{r}=\tilde{R})=\textstyle\frac12\ln\left(1-\frac{2M}{\tilde{R}}\right).
\end{align}
Here, $P(\tilde{r})$ and $\epsilon(\tilde{r})$ are the pressure and total energy density of the Neutron-Star matter.  We have assumed that the matter is a *Perfect Fluid* with a stress-energy tensor given by
\begin{align}
    T^{\mu\nu} = (\epsilon +P)u^\mu u^\nu + P g^{\mu\nu},
\end{align}
where $u^\mu$ is the matter 4-velocity, and $g^{\mu\nu}$ is the inverse metric of the space-time.

## Implementation using OrdinaryDiffEq (*part of DifferentialEquations.jl*)

### Tests

In [ ]:
include("./RotNS.jl")
using .RotNS, Plots

In [ ]:
eos = PolytropicEOS(1.0)
sol=TOVSchwarzschild(eos,0.2736,1.0)
println("R = ",sol.SurfaceRadius," : M = ",sol.TotalMass," : P(R) = ",sol.SurfacePressure)

In [ ]:
plot(sol.Solution,idxs=[(0,1)],label=("Mass"),xlabel="r",ylabel="M")

In [ ]:
plot(sol.Solution,idxs=[(0,2)],label=("Pressure"),xlabel="r",ylabel="P",yaxis=:log)

In [ ]:
ΔΦ = 1/2*log(1-2*sol.TotalMass/sol.SurfaceRadius)-sol.Solution[3,end] # need to compute "global" correction
correctΦ(r,Φ) = (r, Φ+ΔΦ) # function to transform Φ
plot(sol.Solution,idxs=(correctΦ,0,3),label=("Potential"),xlabel="r",ylabel="Φ")

### Polytropes

In [ ]:
QSurface=1.0e-10;

In [ ]:
SurfacePressure = QSurface^(3/2)
eos = PolytropicEOS(1/2)
edrange = exp10.(range(-2.0,log10(5),length=200))
sols0 = map((ced)->TOVSchwarzschild(eos,ced,0.8,SurfacePressure=SurfacePressure),edrange);

In [ ]:
SurfacePressure = QSurface^(2)
eos = PolytropicEOS(1)
edrange = exp10.(range(-4.0,log10(1),length=200))
sols1 = map((ced)->TOVSchwarzschild(eos,ced,1.3,SurfacePressure=SurfacePressure),edrange);

In [ ]:
SurfacePressure = QSurface^(5/2)
eos = PolytropicEOS(1.5)
edrange = exp10.(range(-6.0,log10(0.2),length=200))
sols2 = map((ced)->TOVSchwarzschild(eos,ced,18,SurfacePressure=SurfacePressure),edrange);

In [ ]:
SurfacePressure = QSurface^(3)
eos = PolytropicEOS(2)
edrange = exp10.(range(-10.0,log10(0.02),length=200))
sols3 = map((ced)->TOVSchwarzschild(eos,ced,800,SurfacePressure=SurfacePressure),edrange);

In [ ]:
SurfacePressure = QSurface^(13/4)
eos = PolytropicEOS(9/4)
edrange = exp10.(range(-15.0,log10(0.005),length=200))
sols4 = map((ced)->TOVSchwarzschild(eos,ced,4.e+4,SurfacePressure=SurfacePressure),edrange);

In [ ]:
SurfacePressure = QSurface^(7/2)
eos = PolytropicEOS(5/2)
edrange = exp10.(range(-18,log10(0.0005),length=200))
sols5 = map((ced)->TOVSchwarzschild(eos,ced,9.0e+5,SurfacePressure=SurfacePressure),edrange);

In [ ]:
plot(getfield.(sols0,:CentralDensity),getfield.(sols0,:TotalMass),xaxis=:log,label="n=1/2")
plot!(getfield.(sols1,:CentralDensity),getfield.(sols1,:TotalMass),xaxis=:log,label="n=1")
plot!(getfield.(sols2,:CentralDensity),getfield.(sols2,:TotalMass),xaxis=:log,label="n=3/2")
plot!(getfield.(sols3,:CentralDensity),getfield.(sols3,:TotalMass),xaxis=:log,label="n=2")
plot!(getfield.(sols4,:CentralDensity),getfield.(sols4,:TotalMass),xaxis=:log,label="n=9/4")
plot!(getfield.(sols5,:CentralDensity),getfield.(sols5,:TotalMass),xaxis=:log,label="n=5/2",legend=:topleft);xlabel!("ϵ₀");ylabel!("M")

In [ ]:
plot(getfield.(sols0,:CentralDensity),getfield.(sols0,:SurfaceRadius),xaxis=:log,yaxis=:log,label="n=1/2")
plot!(getfield.(sols1,:CentralDensity),getfield.(sols1,:SurfaceRadius),xaxis=:log,yaxis=:log,label="n=1")
plot!(getfield.(sols2,:CentralDensity),getfield.(sols2,:SurfaceRadius),xaxis=:log,yaxis=:log,label="n=3/2")
plot!(getfield.(sols3,:CentralDensity),getfield.(sols3,:SurfaceRadius),xaxis=:log,yaxis=:log,label="n=2")
plot!(getfield.(sols4,:CentralDensity),getfield.(sols4,:SurfaceRadius),xaxis=:log,yaxis=:log,label="n=9/4")
plot!(getfield.(sols5,:CentralDensity),getfield.(sols5,:SurfaceRadius),xaxis=:log,yaxis=:log,label="n=5/2",legend=:topright);xlabel!("ϵ₀");ylabel!("R")

In [ ]:
default(size = (647, 400))
plot(getfield.(sols0,:SurfaceRadius),getfield.(sols0,:TotalMass),xaxis=:log,label="n=1/2")
plot!(getfield.(sols1,:SurfaceRadius),getfield.(sols1,:TotalMass),xaxis=:log,label="n=1")
plot!(getfield.(sols2,:SurfaceRadius),getfield.(sols2,:TotalMass),xaxis=:log,label="n=3/2")
plot!(getfield.(sols3,:SurfaceRadius),getfield.(sols3,:TotalMass),xaxis=:log,label="n=2")
plot!(getfield.(sols4,:SurfaceRadius),getfield.(sols4,:TotalMass),xaxis=:log,label="n=9/4")
plot!(getfield.(sols5,:SurfaceRadius),getfield.(sols5,:TotalMass),xaxis=:log,label="n=5/2",legend=:topleft);xlabel!("R");ylabel!("M")

### Realistic EOS

In [ ]:
SurfacePressure = 1.01e+10/(Epsilon0*Cspeed^2)

In [ ]:
eos = RealisticEOS("./Tabulated_EOS.h5","A")
edrange = exp10.(range(-0.6,log10(7),length=200))
sols0 = map((ced)->TOVSchwarzschild(eos,ced,2,SurfacePressure=SurfacePressure),edrange);

In [ ]:
eos = RealisticEOS("./Tabulated_EOS.h5","B")
edrange = exp10.(range(-0.6,log10(9),length=200))
sols1 = map((ced)->TOVSchwarzschild(eos,ced,2,SurfacePressure=SurfacePressure),edrange);

In [ ]:
eos = RealisticEOS("./Tabulated_EOS.h5","C")
edrange = exp10.(range(-0.84,log10(6),length=200))
sols2 = map((ced)->TOVSchwarzschild(eos,ced,2,SurfacePressure=SurfacePressure),edrange);

In [ ]:
eos = RealisticEOS("./Tabulated_EOS.h5","D")
edrange = exp10.(range(-0.73,log10(6),length=200))
sols3 = map((ced)->TOVSchwarzschild(eos,ced,2,SurfacePressure=SurfacePressure),edrange);

In [ ]:
eos = RealisticEOS("./Tabulated_EOS.h5","E")
edrange = exp10.(range(-0.62,log10(6),length=200))
sols4 = map((ced)->TOVSchwarzschild(eos,ced,2,SurfacePressure=SurfacePressure),edrange);

In [ ]:
eos = RealisticEOS("./Tabulated_EOS.h5","F")
edrange = exp10.(range(-0.69,log10(10),length=200))
sols5 = map((ced)->TOVSchwarzschild(eos,ced,2,SurfacePressure=SurfacePressure),edrange);

In [ ]:
eos = RealisticEOS("./Tabulated_EOS.h5","G")
edrange = exp10.(range(-0.61,log10(12),length=200))
sols6 = map((ced)->TOVSchwarzschild(eos,ced,2,SurfacePressure=SurfacePressure),edrange);

In [ ]:
eos = RealisticEOS("./Tabulated_EOS.h5","L")
edrange = exp10.(range(-0.82,log10(3),length=200))
sols7 = map((ced)->TOVSchwarzschild(eos,ced,2,SurfacePressure=SurfacePressure),edrange);

In [ ]:
eos = RealisticEOS("./Tabulated_EOS.h5","M")
edrange = exp10.(range(-1.1,log10(3),length=200))
sols8 = map((ced)->TOVSchwarzschild(eos,ced,2,SurfacePressure=SurfacePressure),edrange);

In [ ]:
eos = RealisticEOS("./Tabulated_EOS.h5","N")
edrange = exp10.(range(-0.72,log10(3),length=200))
sols9 = map((ced)->TOVSchwarzschild(eos,ced,2,SurfacePressure=SurfacePressure),edrange);

In [ ]:
eos = RealisticEOS("./Tabulated_EOS.h5","AU")
edrange = exp10.(range(-0.55,log10(5),length=200))
sols10 = map((ced)->TOVSchwarzschild(eos,ced,2,SurfacePressure=SurfacePressure),edrange);

In [ ]:
eos = RealisticEOS("./Tabulated_EOS.h5","UU")
edrange = exp10.(range(-0.615,log10(5),length=200))
sols11 = map((ced)->TOVSchwarzschild(eos,ced,2,SurfacePressure=SurfacePressure),edrange);

In [ ]:
eos = RealisticEOS("./Tabulated_EOS.h5","UT")
edrange = exp10.(range(-0.57,log10(5),length=200))
sols12 = map((ced)->TOVSchwarzschild(eos,ced,2,SurfacePressure=SurfacePressure),edrange);

In [ ]:
eos = RealisticEOS("./Tabulated_EOS.h5","FPS")
edrange = exp10.(range(-0.592,log10(7),length=200))
sols13 = map((ced)->TOVSchwarzschild(eos,ced,2,SurfacePressure=SurfacePressure),edrange);

In [ ]:
default(size = (647, 400))
plot(getfield.(sols0,:CentralDensity),getfield.(sols0,:TotalMass),xaxis=:log,label="A")
plot!(getfield.(sols1,:CentralDensity),getfield.(sols1,:TotalMass),xaxis=:log,label="B")
plot!(getfield.(sols2,:CentralDensity),getfield.(sols2,:TotalMass),xaxis=:log,label="C")
plot!(getfield.(sols3,:CentralDensity),getfield.(sols3,:TotalMass),xaxis=:log,label="D")
plot!(getfield.(sols4,:CentralDensity),getfield.(sols4,:TotalMass),xaxis=:log,label="E")
plot!(getfield.(sols5,:CentralDensity),getfield.(sols5,:TotalMass),xaxis=:log,label="F")
plot!(getfield.(sols6,:CentralDensity),getfield.(sols6,:TotalMass),xaxis=:log,label="G")
plot!(getfield.(sols7,:CentralDensity),getfield.(sols7,:TotalMass),xaxis=:log,label="L") # High Mass
plot!(getfield.(sols8,:CentralDensity),getfield.(sols8,:TotalMass),xaxis=:log,label="M")
plot!(getfield.(sols9,:CentralDensity),getfield.(sols9,:TotalMass),xaxis=:log,label="N") # High Mass
plot!(getfield.(sols10,:CentralDensity),getfield.(sols10,:TotalMass),xaxis=:log,label="AU") # Large Radius at low density
plot!(getfield.(sols11,:CentralDensity),getfield.(sols11,:TotalMass),xaxis=:log,label="UU")
plot!(getfield.(sols12,:CentralDensity),getfield.(sols12,:TotalMass),xaxis=:log,label="UT")
plot!(getfield.(sols13,:CentralDensity),getfield.(sols13,:TotalMass),xaxis=:log,label="FPS",legend=:topleft);xlabel!("ϵ₀");ylabel!("M")

In [ ]:
plot(getfield.(sols0,:CentralDensity),getfield.(sols0,:SurfaceRadius),xaxis=:log,label="A")
plot!(getfield.(sols1,:CentralDensity),getfield.(sols1,:SurfaceRadius),xaxis=:log,label="B")
plot!(getfield.(sols2,:CentralDensity),getfield.(sols2,:SurfaceRadius),xaxis=:log,label="C")
plot!(getfield.(sols3,:CentralDensity),getfield.(sols3,:SurfaceRadius),xaxis=:log,label="D")
plot!(getfield.(sols4,:CentralDensity),getfield.(sols4,:SurfaceRadius),xaxis=:log,label="E")
plot!(getfield.(sols5,:CentralDensity),getfield.(sols5,:SurfaceRadius),xaxis=:log,label="F")
plot!(getfield.(sols6,:CentralDensity),getfield.(sols6,:SurfaceRadius),xaxis=:log,label="G")
plot!(getfield.(sols7,:CentralDensity),getfield.(sols7,:SurfaceRadius),xaxis=:log,label="L")
plot!(getfield.(sols8,:CentralDensity),getfield.(sols8,:SurfaceRadius),xaxis=:log,label="M")
plot!(getfield.(sols9,:CentralDensity),getfield.(sols9,:SurfaceRadius),xaxis=:log,label="N")
plot!(getfield.(sols10,:CentralDensity),getfield.(sols10,:SurfaceRadius),xaxis=:log,label="AU")
plot!(getfield.(sols11,:CentralDensity),getfield.(sols11,:SurfaceRadius),xaxis=:log,label="UU")
plot!(getfield.(sols12,:CentralDensity),getfield.(sols12,:SurfaceRadius),xaxis=:log,label="UT")
plot!(getfield.(sols13,:CentralDensity),getfield.(sols13,:SurfaceRadius),xaxis=:log,label="FPS",legend=:topright);xlabel!("ϵ₀");ylabel!("R")

In [ ]:
plot(getfield.(sols0,:SurfaceRadius),getfield.(sols0,:TotalMass),xaxis=:log,label="A")
plot!(getfield.(sols1,:SurfaceRadius),getfield.(sols1,:TotalMass),xaxis=:log,label="B")
plot!(getfield.(sols2,:SurfaceRadius),getfield.(sols2,:TotalMass),xaxis=:log,label="C")
plot!(getfield.(sols3,:SurfaceRadius),getfield.(sols3,:TotalMass),xaxis=:log,label="D")
plot!(getfield.(sols4,:SurfaceRadius),getfield.(sols4,:TotalMass),xaxis=:log,label="E")
plot!(getfield.(sols5,:SurfaceRadius),getfield.(sols5,:TotalMass),xaxis=:log,label="F")
plot!(getfield.(sols6,:SurfaceRadius),getfield.(sols6,:TotalMass),xaxis=:log,label="G")
plot!(getfield.(sols7,:SurfaceRadius),getfield.(sols7,:TotalMass),xaxis=:log,label="L")
plot!(getfield.(sols8,:SurfaceRadius),getfield.(sols8,:TotalMass),xaxis=:log,label="M")
plot!(getfield.(sols9,:SurfaceRadius),getfield.(sols9,:TotalMass),xaxis=:log,label="N")
plot!(getfield.(sols10,:SurfaceRadius),getfield.(sols10,:TotalMass),xaxis=:log,label="AU") # Large Radius at low density
plot!(getfield.(sols11,:SurfaceRadius),getfield.(sols11,:TotalMass),xaxis=:log,label="UU")
plot!(getfield.(sols12,:SurfaceRadius),getfield.(sols12,:TotalMass),xaxis=:log,label="UT")
plot!(getfield.(sols13,:SurfaceRadius),getfield.(sols13,:TotalMass),xaxis=:log,label="FPS",legend=:topright);xlabel!("R");ylabel!("M")

## Stationary Axisymmetric Metric

A general stationary, axisymmetric metric can be written in an isotropic-like radial coordinate as:
\begin{align}
    \rm{d}s^2 &= -e^{\gamma(r,\theta)+\rho(r,\theta)}{\rm d}t^2 + e^{2\alpha(r,\theta)}({\rm d}r^2+r^2{\rm d}\theta^2)+e^{\gamma(r,\theta)-\rho(r,\theta)}r^2\sin^2\theta({\rm d}\phi-\omega(r,\theta){\rm d}t)^2.
\end{align}
In the static, spherically symmetric limit, this metric takes the form
\begin{align}
    \rm{d}s^2 &= -e^{2\beta(r)}{\rm d}t^2 + e^{2\alpha(r)}({\rm d}r^2+r^2{\rm d}^2\Omega),
\end{align}
where $2\beta(r) = \gamma(r)+\rho(r)$, $2\alpha(r) = \gamma(r)-\rho(r)$, $\omega(r)=0$, and $r$ is a true isotropic radial coordinate.

It is necessary to re-express the metric for the spherically symmetric Neutron Star written in terms of the Schwarzschild radial coordinate $\tilde{r}$ in terms of the isotropic radial coordinate $r$.  This transformation is defined by the relations:
\begin{align}
    r e^{\alpha(r)} &= \tilde{r}, \\
    e^{\alpha(r)}{\rm d}r &= \left(1 - \frac{2m(\tilde{r})}{\tilde{r}}\right)^{-\frac12}{\rm d}\tilde{r}, \\
    \beta(r) &= \Phi(\tilde{r}(r)),
\end{align}
where $m(\tilde{r})=M$ for $\tilde{r}\ge\tilde{R}$.

For $\tilde{r}\ge\tilde{R}$, the transformation can be integrated analytically.
\begin{align}
    e^{\alpha(r)}{\rm d}r = \frac{\tilde{r}}{r}{\rm d}r = \left(1 - \frac{2M}{\tilde{r}}\right)^{-\frac12}{\rm d}\tilde{r}
    \quad\Rightarrow\quad
    \frac{{\rm d}r}{r} = \frac{{\rm d}\tilde{r}}{\tilde{r}\sqrt{1 - \frac{2M}{\tilde{r}}}}
    \quad\Rightarrow\quad
    r = C\left[\frac{1 + \sqrt{1 - \frac{2M}{\tilde{r}}}}{1-\sqrt{1 - \frac{2M}{\tilde{r}}}}\right]
    \quad\Rightarrow\quad
    \tilde{r} = \frac{M}{2C}r\left(1 + \frac{C}{r}\right)^2,
\end{align}
where $C$ is an integration constant.  To fix this integration constant, we not the the metric must be asymptotically flat.  This requires that
\begin{align}
\lim_{r\to\infty}e^{\alpha(r)} = \lim_{r\to\infty}\frac{\tilde{r}}{r} = \lim_{r\to\infty}\frac{M}{2C}\left(1 + \frac{C}{r}\right)^2    
= \frac{M}{2C} \to 1.
\end{align}
So, we find that $C=\frac{M}2$, which yields for $\tilde{r}\ge\tilde{R}$: 
\begin{align}
    \tilde{r} &= r\left(1 + \frac{M}{2r}\right)^2, \\
    e^{2\alpha(r)} &= \left(1 + \frac{M}{2r}\right)^4, \\
    e^{2\beta(r)} &= \left(\frac{1-\frac{M}{2r}}{1+\frac{M}{2r}}\right)^2.
\end{align}

The surface of the Neutron Star is at $\tilde{r}=\tilde{R}$, which corresponds to
\begin{align}
    R =  \frac{M}2\left[\frac{1 + \sqrt{1 - \frac{2M}{\tilde{R}}}}{1-\sqrt{1 - \frac{2M}{\tilde{R}}}}\right]
    = \frac{\tilde{R}}4\left(1 + \sqrt{1 - \frac{2M}{\tilde{R}}}\right)^2
    \qquad\leftrightarrow\qquad
    \tilde{R} = R\left(1+\frac{M}{2R}\right)^2.
\end{align}

Interior to the surface of the Neutron Star, we must transform the TOV equation to use the isotropic radial coordinate and explicitly integrate from the surface of the Neutron Star to the center in order to determine the metric functions $\alpha(r)$ and $\beta(r)$ (*from which we can obtain $\gamma(r)$ and $\rho(r)$*).
\begin{align}
    \frac{{\rm d}\tilde{r}(r)}{{\rm d}r} &= \frac{\tilde{r}}{r}\sqrt{1 - \frac{2m(r)}{\tilde{r}}} &&\text{with}\qquad \tilde{r}(R)=\tilde{R}, \\
    \frac{{\rm d}\beta(r)}{{\rm d}r} &= \frac{{\rm d}\Phi(\tilde{r}(r))}{{\rm d}\tilde{r}}\frac{{\rm d}\tilde{r}}{{\rm d}r}
    = \frac{m(r)+4\pi \tilde{r}^3P(r)}{\tilde{r}[\tilde{r}-2m(r)]}\frac{\tilde{r}}{r}\sqrt{1 - \frac{2m(r)}{\tilde{r}}}
    = \frac{m(r)+4\pi \tilde{r}^3P(r)}{\tilde{r}r\sqrt{1 - \frac{2m(r)}{\tilde{r}}}} &&\text{with}\qquad \beta(R)=\ln\left(\frac{1-\tfrac{M}{2R}}{1+\tfrac{M}{2R}}\right), \\
    \frac{{\rm d}P(r)}{{\rm d}r} &= \frac{{\rm d}P(\tilde{r}(r))}{{\rm d}\tilde{r}}\frac{{\rm d}\tilde{r}}{{\rm d}r}
    = -\frac{[\epsilon(r)+P(r)][m(r)+4\pi \tilde{r}^3P(r)]}{\tilde{r}[\tilde{r}-2m(r)]}\frac{\tilde{r}}{r}\sqrt{1 - \frac{2m(r)}{\tilde{r}}}
    = -[\epsilon(r)+P(r)]\frac{{\rm d}\beta(r)}{{\rm d}r} &&\text{with}\qquad P(R)=0, \\
    \frac{{\rm d}m(r)}{{\rm d}r} &= \frac{{\rm d}m(\tilde{r}(r))}{{\rm d}\tilde{r}}\frac{{\rm d}\tilde{r}}{{\rm d}r}
    = 4\pi\epsilon(r)\frac{\tilde{r}^3}{r}\sqrt{1 - \frac{2m(r)}{\tilde{r}}} &&\text{with}\qquad m(R)=M.
\end{align}
Then:
\begin{align}
    \alpha(r) &= \ln\left(\frac{\tilde{r}}{r}\right), \\
    \gamma(r) &= \beta(r) + \alpha(r), \\
    \rho(r) &= \beta(r) - \alpha(r), \\
    \omega(r) &= 0.
\end{align}

### Solution Scheme

The general solution scheme is to begin by integrating the TOV eqauation in standard Schwarzschild radial coordinates out from $\tilde{r}=0$ to the radius $\tilde{R}$ where $P(\tilde{R})=0$. This is needed to obtain the mass $M$ and radius $\tilde{R}$ of a given Neutron Star model.  The integration is non-standard since the radius at which to end the integration is unknown.  This requires special care to properly terminate the integration and accurately determine $\tilde{R}$.

Once $M$ and $\tilde{R}$ have been determined, we need to integrate again to obtain the metric variables in isotropic radial coordinates.  However, sufficient boundary conditions exist to integrate outward from $r=0$.  Integrating inward from the Neutron Star surface $r=R$ toward the coordinate singularity at $r=0$ is unstable and becomes inaccurate near the origin.  One approach is to integrate to a set of radii in the neighborhood of $r=0$ and then extrapolate the solution to $r=0$ using the fact that the metric potentials will behave as:
\begin{align}
    \tilde{r}(r) &\approx C_1r, \\
    m(r) &\approx C_2r^3, \\
    P(r) &\approx C_3 + C_4r^2, \\
    \beta(r) &\approx C_5 + C_6r^2.
\end{align}
Another difficulty in integrating the TOV equations inward is that we cannot start exactly at the surface $r=R$ of the Neutron Star where the pressure vanshes.  Because $P=0 \leftrightarrow \epsilon=0$, we find that ${\rm d}P/{\rm d}r={\rm d}m/{\rm d}r=0$ at the surface.  This leads to $P(r)$ and $m(r)$ being constant as we integrate in.  In practice, we need to define the surface of the Neutron Star to have a small, but non-zero pressure and density.

An effective scheme to solve the TOV equations in isotropic radial coordinates is to use shooting based on the value of $\left.\frac{{\rm d}\tilde{r}}{{\rm d}r}\right|_{r=0}$.  Together with $\tilde{r}(0)=0$, $m(0)=0$, a set value for $P(0)$, and letting $\beta(0)=0$, the equations can be integrated out to $r=R$ where $P(R)$ is determined.  The valule of $\left.\frac{{\rm d}\tilde{r}}{{\rm d}r}\right|_{r=0}$ is adjusted until $P(R)=0$.  In practice, it is useful to integrate inward first to a small value of $r$ to obtain a good initial guess for $\left.\frac{{\rm d}\tilde{r}}{{\rm d}r}\right|_{r=0}$.  Also, shooting based on $P(R)=0$ has proven to be supperior to using $m(R)=M$ or $\tilde{r}(R)=\tilde{R}$.

## Implementation using OrdinaryDiffEq (*part of DifferentialEquations.jl*)

### Polytropes

In [ ]:
eos = PolytropicEOS(1/2)
edrange = exp10.(range(-2,log10(5),length=200))
sols0 = map((ced)->TOVModel(eos,ced,0.8),edrange);

In [ ]:
eos = PolytropicEOS(1)
edrange = exp10.(range(-4.0,log10(1),length=200))
sols1 = map((ced)->TOVModel(eos,ced,1.3),edrange);

In [ ]:
eos = PolytropicEOS(3/2)
edrange = exp10.(range(-6.0,log10(0.2),length=200))
sols2 = map((ced)->TOVModel(eos,ced,18),edrange);

In [ ]:
eos = PolytropicEOS(2)
edrange = exp10.(range(-10.0,log10(0.02),length=200))
sols3 = map((ced)->TOVModel(eos,ced,800),edrange);

In [ ]:
eos = PolytropicEOS(9/4)
edrange = exp10.(range(-15.0,log10(0.005),length=200))
sols4 = map((ced)->TOVModel(eos,ced,4.e+4),edrange);

In [ ]:
eos = PolytropicEOS(5/2)
edrange = exp10.(range(-18,log10(0.0005),length=200))
sols5 = map((ced)->TOVModel(eos,ced,9.0e+5),edrange);

In [ ]:
default(size = (647, 400))
plot(getfield.(sols0,:CentralDensity),getfield.(sols0,:TotalMass),xaxis=:log,label="n=1/2")
plot!(getfield.(sols1,:CentralDensity),getfield.(sols1,:TotalMass),xaxis=:log,label="n=1")
plot!(getfield.(sols2,:CentralDensity),getfield.(sols2,:TotalMass),xaxis=:log,label="n=3/2")
plot!(getfield.(sols3,:CentralDensity),getfield.(sols3,:TotalMass),xaxis=:log,label="n=2")
plot!(getfield.(sols4,:CentralDensity),getfield.(sols4,:TotalMass),xaxis=:log,label="n=9/4")
plot!(getfield.(sols5,:CentralDensity),getfield.(sols5,:TotalMass),xaxis=:log,label="n=5/2",legend=:topleft);xlabel!("ϵ₀");ylabel!("M")

In [ ]:
default(size = (647, 400))
plot(getfield.(sols0,:CentralDensity),getfield.(sols0,:SurfaceRadius),xaxis=:log,yaxis=:log,label="n=1/2")
plot!(getfield.(sols1,:CentralDensity),getfield.(sols1,:SurfaceRadius),xaxis=:log,yaxis=:log,label="n=1")
plot!(getfield.(sols2,:CentralDensity),getfield.(sols2,:SurfaceRadius),xaxis=:log,yaxis=:log,label="n=3/2")
plot!(getfield.(sols3,:CentralDensity),getfield.(sols3,:SurfaceRadius),xaxis=:log,yaxis=:log,label="n=2")
plot!(getfield.(sols4,:CentralDensity),getfield.(sols4,:SurfaceRadius),xaxis=:log,yaxis=:log,label="n=9/4")
plot!(getfield.(sols5,:CentralDensity),getfield.(sols5,:SurfaceRadius),xaxis=:log,yaxis=:log,label="n=5/2",legend=:topright);xlabel!("ϵ₀");ylabel!("R")

In [ ]:
default(size = (647, 400))
plot(getfield.(sols0,:SurfaceRadius),getfield.(sols0,:TotalMass),xaxis=:log,label="n=1/2")
plot!(getfield.(sols1,:SurfaceRadius),getfield.(sols1,:TotalMass),xaxis=:log,label="n=1")
plot!(getfield.(sols2,:SurfaceRadius),getfield.(sols2,:TotalMass),xaxis=:log,label="n=3/2")
plot!(getfield.(sols3,:SurfaceRadius),getfield.(sols3,:TotalMass),xaxis=:log,label="n=2")
plot!(getfield.(sols4,:SurfaceRadius),getfield.(sols4,:TotalMass),xaxis=:log,label="n=9/4")
plot!(getfield.(sols5,:SurfaceRadius),getfield.(sols5,:TotalMass),xaxis=:log,label="n=5/2",legend=:topleft);xlabel!("R");ylabel!("M")

### Realistic EOS

In [ ]:
eos = RealisticEOS("./Tabulated_EOS.h5","A")
edrange = exp10.(range(-0.6,log10(7),length=200))
sols0 = map((ced)->TOVModel(eos,ced,2),edrange);

In [ ]:
eos = RealisticEOS("./Tabulated_EOS.h5","B")
edrange = exp10.(range(-0.6,log10(9),length=200))
sols1 = map((ced)->TOVModel(eos,ced,2),edrange);

In [ ]:
eos = RealisticEOS("./Tabulated_EOS.h5","C")
edrange = exp10.(range(-0.84,log10(6),length=200))
sols2 = map((ced)->TOVModel(eos,ced,2),edrange);

In [ ]:
eos = RealisticEOS("./Tabulated_EOS.h5","D")
edrange = exp10.(range(-0.73,log10(6),length=200))
sols3 = map((ced)->TOVModel(eos,ced,2),edrange);

In [ ]:
eos = RealisticEOS("./Tabulated_EOS.h5","E")
edrange = exp10.(range(-0.62,log10(6),length=200))
sols4 = map((ced)->TOVModel(eos,ced,2),edrange);

In [ ]:
eos = RealisticEOS("./Tabulated_EOS.h5","F")
edrange = exp10.(range(-0.69,log10(10),length=200))
sols5 = map((ced)->TOVModel(eos,ced,2),edrange);

In [ ]:
eos = RealisticEOS("./Tabulated_EOS.h5","G")
edrange = exp10.(range(-0.61,log10(12),length=200))
sols6 = map((ced)->TOVModel(eos,ced,2),edrange);

In [ ]:
eos = RealisticEOS("./Tabulated_EOS.h5","L")
edrange = exp10.(range(-0.82,log10(3),length=200))
sols7 = map((ced)->TOVModel(eos,ced,2),edrange);

In [ ]:
eos = RealisticEOS("./Tabulated_EOS.h5","M")
edrange = exp10.(range(-1.1,log10(3),length=200))
sols8 = map((ced)->TOVModel(eos,ced,2),edrange);

In [ ]:
eos = RealisticEOS("./Tabulated_EOS.h5","N")
edrange = exp10.(range(-0.72,log10(3),length=200))
sols9 = map((ced)->TOVModel(eos,ced,2),edrange);

In [ ]:
eos = RealisticEOS("./Tabulated_EOS.h5","AU")
edrange = exp10.(range(-0.48,log10(5),length=200))
sols10 = map((ced)->TOVModel(eos,ced,2),edrange);

In [ ]:
eos = RealisticEOS("./Tabulated_EOS.h5","UU")
edrange = exp10.(range(-0.54,log10(5),length=200))
sols11 = map((ced)->TOVModel(eos,ced,2),edrange);

In [ ]:
eos = RealisticEOS("./Tabulated_EOS.h5","UT")
edrange = exp10.(range(-0.52,log10(5),length=200))
sols12 = map((ced)->TOVModel(eos,ced,2),edrange);

In [ ]:
eos = RealisticEOS("./Tabulated_EOS.h5","FPS")
edrange = exp10.(range(-0.59,log10(7),length=200))
sols13 = map((ced)->TOVModel(eos,ced,2),edrange);

In [ ]:
default(size = (647, 400))
plot(getfield.(sols0,:CentralDensity),getfield.(sols0,:TotalMass),xaxis=:log,label="A")
plot!(getfield.(sols1,:CentralDensity),getfield.(sols1,:TotalMass),xaxis=:log,label="B")
plot!(getfield.(sols2,:CentralDensity),getfield.(sols2,:TotalMass),xaxis=:log,label="C")
plot!(getfield.(sols3,:CentralDensity),getfield.(sols3,:TotalMass),xaxis=:log,label="D")
plot!(getfield.(sols4,:CentralDensity),getfield.(sols4,:TotalMass),xaxis=:log,label="E")
plot!(getfield.(sols5,:CentralDensity),getfield.(sols5,:TotalMass),xaxis=:log,label="F")
plot!(getfield.(sols6,:CentralDensity),getfield.(sols6,:TotalMass),xaxis=:log,label="G")
plot!(getfield.(sols7,:CentralDensity),getfield.(sols7,:TotalMass),xaxis=:log,label="L") # High Mass
plot!(getfield.(sols8,:CentralDensity),getfield.(sols8,:TotalMass),xaxis=:log,label="M")
plot!(getfield.(sols9,:CentralDensity),getfield.(sols9,:TotalMass),xaxis=:log,label="N") # High Mass
plot!(getfield.(sols10,:CentralDensity),getfield.(sols10,:TotalMass),xaxis=:log,label="AU") # Large Radius at low density
plot!(getfield.(sols11,:CentralDensity),getfield.(sols11,:TotalMass),xaxis=:log,label="UU")
plot!(getfield.(sols12,:CentralDensity),getfield.(sols12,:TotalMass),xaxis=:log,label="UT")
plot!(getfield.(sols13,:CentralDensity),getfield.(sols13,:TotalMass),xaxis=:log,label="FPS",legend=:topleft);xlabel!("ϵ₀");ylabel!("M")

In [ ]:
default(size = (647, 400))
plot(getfield.(sols0,:CentralDensity),getfield.(sols0,:SurfaceRadius),xaxis=:log,label="A")
plot!(getfield.(sols1,:CentralDensity),getfield.(sols1,:SurfaceRadius),xaxis=:log,label="B")
plot!(getfield.(sols2,:CentralDensity),getfield.(sols2,:SurfaceRadius),xaxis=:log,label="C")
plot!(getfield.(sols3,:CentralDensity),getfield.(sols3,:SurfaceRadius),xaxis=:log,label="D")
plot!(getfield.(sols4,:CentralDensity),getfield.(sols4,:SurfaceRadius),xaxis=:log,label="E")
plot!(getfield.(sols5,:CentralDensity),getfield.(sols5,:SurfaceRadius),xaxis=:log,label="F")
plot!(getfield.(sols6,:CentralDensity),getfield.(sols6,:SurfaceRadius),xaxis=:log,label="G")
plot!(getfield.(sols7,:CentralDensity),getfield.(sols7,:SurfaceRadius),xaxis=:log,label="L")
plot!(getfield.(sols8,:CentralDensity),getfield.(sols8,:SurfaceRadius),xaxis=:log,label="M")
plot!(getfield.(sols9,:CentralDensity),getfield.(sols9,:SurfaceRadius),xaxis=:log,label="N")
plot!(getfield.(sols10,:CentralDensity),getfield.(sols10,:SurfaceRadius),xaxis=:log,label="AU")
plot!(getfield.(sols11,:CentralDensity),getfield.(sols11,:SurfaceRadius),xaxis=:log,label="UU")
plot!(getfield.(sols12,:CentralDensity),getfield.(sols12,:SurfaceRadius),xaxis=:log,label="UT")
plot!(getfield.(sols13,:CentralDensity),getfield.(sols13,:SurfaceRadius),xaxis=:log,label="FPS",legend=:topright);xlabel!("ϵ₀");ylabel!("R")

In [ ]:
default(size = (647, 400))
plot(getfield.(sols0,:SurfaceRadius),getfield.(sols0,:TotalMass),xaxis=:log,label="A")
plot!(getfield.(sols1,:SurfaceRadius),getfield.(sols1,:TotalMass),xaxis=:log,label="B")
plot!(getfield.(sols2,:SurfaceRadius),getfield.(sols2,:TotalMass),xaxis=:log,label="C")
plot!(getfield.(sols3,:SurfaceRadius),getfield.(sols3,:TotalMass),xaxis=:log,label="D")
plot!(getfield.(sols4,:SurfaceRadius),getfield.(sols4,:TotalMass),xaxis=:log,label="E")
plot!(getfield.(sols5,:SurfaceRadius),getfield.(sols5,:TotalMass),xaxis=:log,label="F")
plot!(getfield.(sols6,:SurfaceRadius),getfield.(sols6,:TotalMass),xaxis=:log,label="G")
plot!(getfield.(sols7,:SurfaceRadius),getfield.(sols7,:TotalMass),xaxis=:log,label="L")
plot!(getfield.(sols8,:SurfaceRadius),getfield.(sols8,:TotalMass),xaxis=:log,label="M")
plot!(getfield.(sols9,:SurfaceRadius),getfield.(sols9,:TotalMass),xaxis=:log,label="N")
plot!(getfield.(sols10,:SurfaceRadius),getfield.(sols10,:TotalMass),xaxis=:log,label="AU") # Large Radius at low density
plot!(getfield.(sols11,:SurfaceRadius),getfield.(sols11,:TotalMass),xaxis=:log,label="UU")
plot!(getfield.(sols12,:SurfaceRadius),getfield.(sols12,:TotalMass),xaxis=:log,label="UT")
plot!(getfield.(sols13,:SurfaceRadius),getfield.(sols13,:TotalMass),xaxis=:log,label="FPS",legend=:topright);xlabel!("R");ylabel!("M")

### Tests

In [ ]:
eos = PolytropicEOS(1.0)
sol=TOVModel(eos,0.2736,1.0)
println("R = ",sol.SurfaceRadius," : M = ",sol.TotalMass," : P(R) = ",sol.SurfacePressure)

In [ ]:
plot(sol.Solution,idxs=[(0,1)],label=("Mass"),xlabel="r",ylabel="M")

In [ ]:
plot(sol.Solution,idxs=[(0,3)],label=("Pressure"),xlabel="r",ylabel="P",yaxis=:log)

In [ ]:
Δβ = log(1-2*sol.TotalMass/sol.SurfaceRadius)-log(1+2*sol.TotalMass/sol.SurfaceRadius)-sol.Solution[2,end] # need to compute "global" correction
correctβ(r,β) = (r, β+Δβ) # function to transform Φ
plot(sol.Solution,idxs=(correctβ,0,2),label=("Potential"),xlabel="r",ylabel="β")

## Equations of State

Solutions are parameterized in terms of the central pressure $P_c$, or equivalently in terms of the central entergy density $\epsilon_c$.  The pressure and energy density are related via an Equation of State (EOS) which specifies $\epsilon(P)$, or equivalently $P(\epsilon)$.  The EOS also specifies the rest-mass density $\rho_0$ as a function of either pressure or energy density.  We define the internal energy density $\rho_i$ such that
\begin{align}
    \epsilon = \rho_0 + \rho_i.
\end{align}
For Neutron-Star matter, we will take the rest-mass density to be given by
\begin{align}
    \rho_0 = \mu_0 n_b,
\end{align}
where $\mu_0\approx 1.659\times10^{-24}\,\text{g}/\text{cm}^3$ is the average mass-per-baryon, and $n_b$ is the number density of baryons.  We will consider two types of Equations of State:

### Polytropic Equation of State

For a polytropic equation-of-state (EOS), we have
\begin{align} 
	P &=K\rho_0^{1+\frac1n}, \\
	&= (\Gamma-1)\rho_i,
\end{align}
where the polytropic index $n$ and adiabatic index $\Gamma$ are related by
\begin{align}
	\Gamma = 1 + \frac1n.
\end{align}
Here, $K$ is the polytropic constant, and the relationship between pressure and internal energy density assumes that all changes are adiabatic.  The polytropic index takes the range of values $\frac12\le n<3$, corresponding to a range of $\frac43<\Gamma\le3$ for the adiabatic index.  In general terminology, an EOS is referred to as being "stiffer" for larger values of $\Gamma$ and "softer" for smaller values of $\Gamma$.

### Realistic Equations of State

We will also consider realistic equations of state provided in tabular form.  These tabular EOS are a *fusion* of different equations of state which span a large range of pressure and density.  At low density, we use the Feynman-Metropolis-Teller (FMT) EOS, which then joins onto the Baym-Pethick-Sutherland (BPS) EOS up to neutron drip.  Above neutron drip, most EOS join onto the Baym-Bethe-Pethick (BBP) EOS and then onto a specific high-density EOS.  In some cases, the BBP EOS is replaced by the Negele-Vautherin (NV) EOS.  All tabulated EOS present 3 columns of data:
\begin{align}
    \text{Column 1} &:\quad \text{Baryon number density \ } n_b \text{\ in units of\ }(\text{cm}^{-3}) \\
    \text{Column 2} &:\quad \text{Total mass density \ } \epsilon \text{\ in units of\ }(\text{g}\cdot\text{cm}^{-3}) \\
    \text{Column 3} &:\quad \text{Pressure \ } P \text{\ in units of\ }(\text{dyne}\cdot\text{cm}^{-2})
\end{align}
Some tabulated EOS containt *phase transitions* in the form of multiple consecutive entries with the same value of the pressure.

#### Tabulated EOS Interpolations

Tabulated EOS values are interpolated using linear interpolation: $y(x) = y_{lo} + \frac{(x-x_{lo})}{(x_{hi}-x_{lo})}(y_{hi}-y_{lo})$.  Each column of tabulated data is in non-decreasing order, but allowing for consecutive value to be equal.  During interpolation, values for $x_{lo}$ and $x_{hi}$ are chosen as consecutive values in a column such that $x_{lo}\le x < x_{hi}$, unless there is a phase transition in that column, in which $x_{lo}=x=x_{hi}$ and $x_{lo}$ chosen as the first of the sequential values.  Values for $y_{lo}$ and $y_{hi}$ are taken from the same rows as the corresponding $x_{lo}$ and $x_{hi}$.

## Dimensionless Variables.

During computations, it is useful to use dimensionless variables.  Depending on the EOS, we will define a dimensionfull constant $\kappa$ to have dimensions of $(\text{length})^2$.  For polytropes, we will take $\kappa = K^n$, where $K$ is the polytropic constant.  For realistic EOS, we will take $\kappa = \frac{c^2}{G\epsilon_0}$, where $\epsilon_0=10^{-15}\,\text{g}\cdot\text{cm}^{-3}$.  We will define quantities with "overbars" to be dimensionless quantities.  For example, the radial and time coordinates, including factors of $G$ and $c$, behave as:
\begin{align}
    \bar{r} &\equiv \kappa^{-\frac12}\,r, \\
    \bar{t} &\equiv \kappa^{-\frac12}\,ct.
\end{align}
The rest-mass density and total energy density behave as:
\begin{align}
    \bar\rho_0 &\equiv \kappa\,\frac{G}{c^2}\rho_0, \\
    \bar\epsilon &\equiv \kappa\,\frac{G}{c^2}\epsilon,
\end{align}
*and we note that the total energy density is expressed as mass and not as energy.*  Pressure behaves as:
\begin{align}
    \bar{P} \equiv \kappa\,\frac{G}{c^4}P,
\end{align}
while mass $M$, angular momentum $J$, and angular velocities like $\omega$ and $\Omega$ behave as:
\begin{align}
    \bar{M} &\equiv \kappa^{-\frac12}\,\frac{G}{c^2}M, \\
    \bar{J} &\equiv \kappa^{-1}\,\frac{G}{c^3}J, \\
    \bar\omega &\equiv \kappa^{\frac12}\, \frac1{c}\omega, \\
    \bar\Omega &\equiv \kappa^{\frac12}\, \frac1{c}\Omega.
\end{align}
Finally, for polytopes, an additional dimensionless variable is often useful.  In geometric units:
\begin{align}
    q &\equiv \frac{P}{\rho_0} = \frac{\bar{P}}{\bar\rho_0}.
\end{align}
In terms of $q$, we find that 
\begin{align}
    \bar\rho_0 &\equiv q^n, \\
    \bar\rho_i &\equiv nq^{n+1}, \\
    \bar\epsilon &= (1+nq)q^n, \\
    \bar{P} &\equiv q^{n+1}.
\end{align}